# Notes for Possible Improvements
- Grid estimation could be improved by linear regression after assigning each point to appropriate gridline.
    - Estimation using x,y coords of point centers for each horizontal and vertical grid. This would improve the gridline intersection accuracy on deformations and orientation errors.
    - Would increase complexity with e.g. 256 regressions with 16x16 DMC
    - Possible efficient solution could be to only have 2 or 4 regressions on DMC alignment pattern, and copying their params accross other gridlines.
- Simple guaranteed error correction before final decoding process
    - Force L and timing to be black as they are consistent
- Implementation of decoding instead of using pylibdmtx
    - Have not done yet due to complexity of it (even though it is deterministic algorithms)
    - Could possibly "steal" relevant parts from pylibdmtx or libdmtx and rewrite for this purpose
- Rewrites for squeezing out performance is possible
    - Avoiding later matrix inversion by assigning 1s and 0s in reverse
    - Avoiding multiple sorting by ensuring methods do not reorder points

# Setup

In [ ]:
import cv2
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans

from pylibdmtx.pylibdmtx import decode

## Yucheng Process

## Yucheng Funcs

In [ ]:
def single_scale_retinex(image, sigma=30):
    """
    Does single scale retinex on the input image.

    Args:
        image: Input image (numpy array)
        sigma: Gaussian kernel size (default is 30)
    
    Returns:
        Tuple of reflectance and illumination images
    """
    image = image.astype(np.float32) + 1.0
    illumination = cv2.GaussianBlur(image, (0, 0), sigma)
    illumination += 1.0
    reflectance = np.log(image) - np.log(illumination)
    reflectance_display = cv2.normalize(reflectance, None, 0, 255, cv2.NORM_MINMAX)
    reflectance_display = reflectance_display.astype(np.uint8)
    illumination_display = cv2.normalize(illumination, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)
    return reflectance_display, illumination_display

In [ ]:
def non_max_suppression_fast(boxes, scores, overlap_thresh=0.3):
    """
    Perform non-maximum suppression on the bounding boxes.

    Args:
        boxes: List of bounding boxes (x, y, width, height)
        scores: List of scores for each bounding box
        overlap_thresh: Overlap threshold for suppression (default is 0.3)
    
    Returns:
        List of bounding boxes after non-maximum suppression
    """
    if len(boxes) == 0:
        return []
    boxes = np.array(boxes)
    scores = np.array(scores)
    x1 = boxes[:, 0]
    y1 = boxes[:, 1]
    x2 = boxes[:, 0] + boxes[:, 2]
    y2 = boxes[:, 1] + boxes[:, 3]
    areas = (x2 - x1 + 1) * (y2 - y1 + 1)
    idxs = scores.argsort()[::-1]
    keep = []
    while len(idxs) > 0:
        i = idxs[0]
        keep.append(i)
        xx1 = np.maximum(x1[i], x1[idxs[1:]])
        yy1 = np.maximum(y1[i], y1[idxs[1:]])
        xx2 = np.minimum(x2[i], x2[idxs[1:]])
        yy2 = np.minimum(y2[i], y2[idxs[1:]])
        w = np.maximum(0, xx2 - xx1 + 1)
        h = np.maximum(0, yy2 - yy1 + 1)
        inter = w * h
        overlap = inter / (areas[i] + areas[idxs[1:]] - inter)
        idxs = idxs[1:][overlap < overlap_thresh]
    return boxes[keep]

In [ ]:
def extract_dominant_dot_template(image, min_area=20, max_area=300, patch_size=(24, 24), offset=5, size_tol=0.5):
    """
    Extracts the dominant dot template from the image.
    The function applies a series of image processing techniques to identify and extract the dot template.

    Args:
        image: Input image (numpy array)
        min_area: Minimum area of the dot to be considered (default is 20)
        max_area: Maximum area of the dot to be considered (default is 300)
        patch_size: Size of the patch to be extracted (default is (24, 24))
        offset: Offset for bounding box around the detected dot (default is 5)
        size_tol: Tolerance for size consistency (default is 0.5)

    Returns:
        Tuple of the extracted patch and contours of the detected dots.
    
    Raises:
        ValueError: If no valid dot candidates are found or if no size-consistent patches are found.
    """
    image_clean = cv2.bilateralFilter(image, d=15, sigmaColor=50, sigmaSpace=5)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    image_clean = clahe.apply(image_clean)
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (16, 16))
    tophat = cv2.morphologyEx(image_clean, cv2.MORPH_BLACKHAT, kernel)

    _, binary_top = cv2.threshold(tophat, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    contours, _ = cv2.findContours(binary_top, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    candidates = []
    sizes = []
    img_w, img_h = image.shape

    for cnt in contours:
        area = cv2.contourArea(cnt)
        if min_area < area < max_area:
            x, y, w, h = cv2.boundingRect(cnt)
            crop_x_start = x - offset
            crop_x_end = x + w + offset
            crop_y_start = y - offset
            crop_y_end = y + h + offset

            if crop_x_start < 0 or crop_x_end >= img_w or crop_y_start < 0 or crop_y_end >= img_h:
                continue

            patch = image[crop_y_start:crop_y_end, crop_x_start:crop_x_end]
            candidates.append((patch, h, w))
            sizes.append((h, w))

    if not candidates:
        raise ValueError("No valid dot candidates found.")

    # Compute median size
    heights = [s[0] for s in sizes]
    widths = [s[1] for s in sizes]
    median_area = np.median(heights) * np.median(widths)

    # Keep only patches with similar size
    patches_filtered = []
    resized_for_matching = []
    for (patch, h, w) in candidates:
        # print(abs(h * w - median_area))
        if abs(h * w - median_area) / median_area < size_tol:
            patches_filtered.append(patch)
            resized_for_matching.append(cv2.resize(patch, patch_size))

    if not patches_filtered:
        raise ValueError("No size-consistent patches found.")

    # Find patch closest to the median template
    stack = np.stack(resized_for_matching, axis=0).astype(np.float32)
    median_template = np.median(stack, axis=0)
    diffs = [np.linalg.norm(p.astype(np.float32) - median_template) for p in resized_for_matching]
    best_idx = np.argmin(diffs)

    return patches_filtered[best_idx], contours

In [ ]:
def contours_from_patch(patch):
    """
    Extracts contours from supplied patch image.
    """
    _, binary_patch = cv2.threshold(patch, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    contours, _ = cv2.findContours(binary_patch, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    return contours

In [ ]:
def display_image(image, size=(300, 300)):
    """
    Displays the numpy image using PIL and notebook display functionality.

    Args:
        image: Input image (numpy array)
        size: Size to which the image should be resized (default is (300, 300))
    
    Returns:
        None
    """
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, size)
    pil_image = Image.fromarray(image)
    display(pil_image)

In [ ]:
def display_yucheng_methods(nms_boxes, reflectance, dot_contours, img, illumination, dot_template):
    """
    Displays the results of Yuchengs methods for dot detection and template matching.

    Args:
        nms_boxes: List of bounding boxes after non-maximum suppression
        reflectance: Reflectance map (numpy array)
        dot_contours: Contours of the detected dots
        img: Original image (numpy array)
        illumination: Estimated illumination (numpy array)
        dot_template: Dot template (numpy array)
    
    Returns:
        None
    """
    # === Draw matching result ===
    output = cv2.cvtColor(reflectance, cv2.COLOR_GRAY2BGR)
    for (x, y, w, h) in nms_boxes:
        cv2.rectangle(output, (x, y), (x + w, y + h), (0, 255, 0), 2)

    # === Draw contours over reflectance ===
    contour_vis = cv2.cvtColor(reflectance, cv2.COLOR_GRAY2BGR)
    cv2.drawContours(contour_vis, dot_contours, -1, (0, 0, 255), 1)

    # === Show results ===
    fig, axs = plt.subplots(2, 3, figsize=(10, 6))
    axs[0, 0].imshow(img, cmap='gray')
    axs[0, 0].set_title("Original Image")
    axs[0, 0].axis("off")

    axs[0, 1].imshow(illumination, cmap='gray')
    axs[0, 1].set_title("Estimated Illumination")
    axs[0, 1].axis("off")

    axs[0, 2].imshow(reflectance, cmap='gray')
    axs[0, 2].set_title("Reflectance Map (SSR)")
    axs[0, 2].axis("off")

    axs[1, 0].imshow(cv2.cvtColor(contour_vis, cv2.COLOR_BGR2RGB))
    axs[1, 0].set_title("Dot Contours")
    axs[1, 0].axis("off")

    axs[1, 1].imshow(dot_template, cmap='gray')
    axs[1, 1].set_title("Dot template (median of patches)")
    axs[1, 1].axis("off")

    axs[1, 2].imshow(cv2.cvtColor(output, cv2.COLOR_BGR2RGB))
    axs[1, 2].set_title("Template matching")
    axs[1, 2].axis("off")

    plt.tight_layout()
    plt.show()

## Yucheng Use

In [ ]:
# === Load image (grayscale) ===
img_to_test = "../data/delete.jpg"
template_to_test = "../data/delete_template.jpg"
img = cv2.imread(img_to_test, cv2.IMREAD_GRAYSCALE)
img = cv2.resize(img, (320, 320))
dot_template = cv2.imread(template_to_test, cv2.IMREAD_GRAYSCALE)
dot_template = cv2.resize(dot_template, (19, 19))

In [ ]:
# === Apply Retinex ===
reflectance, illumination = single_scale_retinex(img, sigma=64)

In [ ]:
dot_contours = contours_from_patch(dot_template)

In [ ]:
# # === Extract dominant template and contours ===
# # REPLACE WITH YOUR ACTUAL TEMPLATE
# dot_template, dot_contours = extract_dominant_dot_template(reflectance,
#                                                            min_area=20,
#                                                            max_area=300,
#                                                            patch_size=(24, 24),
#                                                            offset=5,
#                                                            size_tol=0.5)

# display_image(dot_template)

In [ ]:
# === Template matching ===
result = cv2.matchTemplate(reflectance, dot_template, cv2.TM_CCOEFF_NORMED)
threshold = 0.7
locations = zip(*np.where(result >= threshold)[::-1])
scores = result[result >= threshold].flatten()

In [ ]:
# === Bounding boxes (x, y, w, h) for each match ===
h, w = dot_template.shape
boxes = [(int(x), int(y), w, h) for (x, y) in locations]

In [ ]:
# === Apply NMS ===
nms_boxes = non_max_suppression_fast(boxes, scores, overlap_thresh=0.3)

In [ ]:
display_yucheng_methods(nms_boxes, reflectance, dot_contours, img, illumination, dot_template)

# Decoding

## Decoding Funcs

In [ ]:
def grid_estimation(nms_boxes, dmc_size=16):
    """
    Estimate DMC grid from the detected bounding boxes.
    Using KMeans clustering on the x and y coordinates of the boxes to find the grid lines.
    The function assumes that the boxes are aligned in a grid pattern and correct orientation was previously applied.

    Args:
        nms_boxes: List of bounding boxes (x, y, width, height)
        dmc_size: Size of the DMC (default is 16 for 16x16 DMC)
    
    Returns:
        List of estimated grid lines (x_list, y_list)
    """
    x_list = []
    y_list = []

    for box in nms_boxes:
        x, y, w, h = box
        x_list.append(x + w // 2)
        y_list.append(y + h // 2)
    
    x_list = np.array(x_list)
    y_list = np.array(y_list)

    kmeans_x = KMeans(n_clusters=dmc_size, random_state=0).fit(x_list.reshape(-1, 1))
    kmeans_y = KMeans(n_clusters=dmc_size, random_state=0).fit(y_list.reshape(-1, 1))

    x_centers = kmeans_x.cluster_centers_.flatten()
    y_centers = kmeans_y.cluster_centers_.flatten()

    x_centers = np.sort(x_centers)
    y_centers = np.sort(y_centers)

    return x_centers.astype(int), y_centers.astype(int)

In [ ]:
def display_grid(image, x_centers, y_centers):
    """
    Display the grid on the image.

    Args:
        image: Input image (numpy array)
        x_centers: List of x coordinates for grid lines
        y_centers: List of y coordinates for grid lines
    
    Returns:
        None
    """
    for x in x_centers:
        cv2.line(image, (x, 0), (x, image.shape[0]), (255, 0, 0), 1)
    for y in y_centers:
        cv2.line(image, (0, y), (image.shape[1], y), (255, 0, 0), 1)

    plt.imshow(image, cmap='gray')
    plt.title("Grid Estimation")
    plt.axis("off")
    plt.show()

In [ ]:
def get_grid_intersections(x_centers, y_centers):
    """
    Get the grid intersections from the x and y coordinates.

    Args:
        x_centers: List of x coordinates for grid lines
        y_centers: List of y coordinates for grid lines
    
    Returns:
        List of grid intersections (x, y)
    """
    intersections = []
    for x in x_centers:
        for y in y_centers:
            intersections.append((x, y))
    return intersections

In [ ]:
def display_grid_intersections(image, intersections):
    """
    Display the grid intersections on the image.

    Args:
        image: Input image (numpy array)
        intersections: List of grid intersections (x, y)
    
    Returns:
        None
    """
    for (x, y) in intersections:
        cv2.circle(image, (x, y), 3, (0, 255, 0), -1)

    plt.imshow(image, cmap='gray')
    plt.title("Grid Intersections")
    plt.axis("off")
    plt.show()

In [ ]:
def find_closest_intersection(x, y, intersections):
    """
    Find the closest grid intersection to the given coordinates.

    Args:
        x: x coordinate
        y: y coordinate
        intersections: List of grid intersections (x, y)
    
    Returns:
        Closest intersection (x, y)
    """
    closest = None
    min_dist = float('inf')
    for (ix, iy) in intersections:
        dist = np.sqrt((x - ix) ** 2 + (y - iy) ** 2)
        if dist < min_dist:
            min_dist = dist
            closest = (ix, iy)
    return closest

def find_all_closest_intersections(coords, intersections):
    """
    Find the closest grid intersections for a list of coordinates.

    Args:
        coords: List of coordinates (x, y)
        intersections: List of grid intersections (x, y)
    
    Returns:
        List of closest intersections (x, y)
    """
    closest_intersections = []
    for (x, y) in coords:
        x_center = x + w // 2
        y_center = y + h // 2
        closest = find_closest_intersection(x_center, y_center, intersections)
        closest_intersections.append(closest)
    return closest_intersections

def display_closest_intersections(image, nms_boxes, closest_intersections):
    """
    Displays bounding boxes and their closest intersections.
    """
    output = cv2.cvtColor(image, cv2.COLOR_GRAY2BGR)
    for (x, y, w, h), (ix, iy) in zip(nms_boxes, closest_intersections):
        cv2.rectangle(output, (x, y), (x + w, y + h), (0, 255, 0), 2)
        cv2.circle(output, (ix, iy), 3, (255, 0, 0), -1)

    plt.imshow(output)
    plt.title("Closest Intersections")
    plt.axis("off")
    plt.show()

In [ ]:
def get_matrix_map(intersections):
    """
    Builds a mapping of intersections to their underlying matrix indices.

    Args:
        intersections: List of grid intersections (x, y)
    
    Returns:
        Dictionary mapping (x, y) to matrix index
        size of the matrix
    """
    matrix_map = {}
    last_y = None
    row = -1
    col = -1

    # Sort intersections by y (ascending) and x (ascending) so that we loop from top to bottom and left to right
    intersections = sorted(intersections, key=lambda p: (p[1], p[0]))

    for x, y in intersections:
        col += 1
        if y != last_y:
            row += 1
            last_y = y
            col = 0
        matrix_map[(x, y)] = (row, col)

    return matrix_map, row+1

def build_matrix(matrix_map, closest_intersections, dmc_size):
    """
    Build a matrix from the grid intersections and their closest intersections.

    Args:
        matrix_map: Dictionary mapping (x, y) to matrix index
        closest_intersections: List of closest intersections (x, y)
    
    Returns:
        Numpy array representing the matrix with 1s at the closest intersections
        and 0s elsewhere.
    """
    matrix = np.zeros((dmc_size, dmc_size), dtype=np.uint8)

    for (x, y) in closest_intersections:
        x_index, y_index = matrix_map[(x, y)]
        matrix[x_index, y_index] = 1
    return matrix

def display_DMC(matrix):
    """
    Display the DMC matrix like normal DMC.

    Args:
        matrix: Numpy array representing the DMC matrix
    """
    # Invert the matrix for display
    matrix = np.invert(matrix)
    plt.imshow(matrix, cmap='gray', interpolation='nearest')
    plt.title("DMC Matrix")
    plt.axis("off")
    plt.show()

In [ ]:
def decode_DMC(matrix):
    """
    Decodes the DMC matrix using pylibdmtx.

    Args:
        matrix: Numpy array representing the DMC matrix
    
    Returns:
        
    """
    # Converting binary matrix to uint8 image
    image = np.zeros((matrix.shape[0], matrix.shape[1]), dtype=np.uint8)
    image[matrix == 1] = 255
    image = Image.fromarray(image, 'L')

    # Inverting the image for decoding
    image = Image.eval(image, lambda x: 255 - x)

    # Padding the image by 2 pixels to add margin larger than a DMC module (https://www.keyence.eu/ss/products/auto_id/codereader/basic_2d/datamatrix.jsp)
    image = np.pad(np.array(image), ((2, 2), (2, 2)), mode='constant', constant_values=255)
    image = Image.fromarray(image, 'L')

    # Resizing to larger image for better decoding
    image = image.resize((image.size[0] * 10, image.size[1] * 10), Image.NEAREST)

    # Decode using pylibdmtx
    decoded = decode(image)
    if decoded:
        return decoded[0].data.decode('utf-8')
    else:
        return None

## Decoding Use

In [ ]:
# === Estimating Grid ===
x_centers, y_centers = grid_estimation(nms_boxes)
display_grid(reflectance.copy(), x_centers, y_centers)

In [ ]:
# === Get grid intersections ===
intersections = get_grid_intersections(x_centers, y_centers)
display_grid_intersections(reflectance.copy(), intersections)

In [ ]:
# === Find closest intersection ===
closest_intersections = find_all_closest_intersections(nms_boxes[:, :2], intersections)
display_closest_intersections(reflectance.copy(), nms_boxes, closest_intersections)

In [ ]:
# === Convert to original DMC ===
matrix_map, dmc_size = get_matrix_map(intersections)
matrix = build_matrix(matrix_map, closest_intersections, dmc_size)
display_DMC(matrix)

In [ ]:
# === Manually fixing error in example ===
matrix[15, 8] = 1
display_DMC(matrix)

In [ ]:
# === Decoding DMC ===
decoded_data = decode_DMC(matrix)
print(decoded_data)

# Full Decoding Pipeline

In [ ]:
def decode_pipeline(image_path, template_path, debug=False):
    """
    Performs the entire decoding pipeline on the input image and template.

    Args:
        image_path: Path to the input image
        template_path: Path to the template image
    
    Returns:
        Decoded data from the DMC matrix or None if decoding fails.
    """
    # === Load image (grayscale) ===
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    img = cv2.resize(img, (320, 320))
    dot_template = cv2.imread(template_path, cv2.IMREAD_GRAYSCALE)
    dot_template = cv2.resize(dot_template, (19, 19))

    # === Apply Retinex ===
    reflectance, illumination = single_scale_retinex(img, sigma=64)

    # === Extract dot contours ===
    dot_contours = contours_from_patch(dot_template)

    # === Template matching ===
    result = cv2.matchTemplate(reflectance, dot_template, cv2.TM_CCOEFF_NORMED)
    threshold = 0.7
    locations = zip(*np.where(result >= threshold)[::-1])
    scores = result[result >= threshold].flatten()

    # === Bounding boxes (x, y, w, h) for each match ===
    h, w = dot_template.shape
    boxes = [(int(x), int(y), w, h) for (x, y) in locations]

    # === Apply NMS ===
    nms_boxes = non_max_suppression_fast(boxes, scores, overlap_thresh=0.3)

    if debug:
        display_yucheng_methods(nms_boxes, reflectance, dot_contours, img, illumination, dot_template)

    # === Estimating Grid ===
    x_centers, y_centers = grid_estimation(nms_boxes)
    if debug:
        display_grid(reflectance.copy(), x_centers, y_centers)

    # === Get grid intersections ===
    intersections = get_grid_intersections(x_centers, y_centers)
    if debug:
        display_grid_intersections(reflectance.copy(), intersections)

    # === Find closest intersection ===
    closest_intersections = find_all_closest_intersections(nms_boxes[:, :2], intersections)
    if debug:
        display_closest_intersections(reflectance.copy(), nms_boxes, closest_intersections)

    # === Convert to original DMC ===
    matrix_map, dmc_size = get_matrix_map(intersections)
    matrix = build_matrix(matrix_map, closest_intersections, dmc_size)
    if debug:
        display_DMC(matrix)

    # === Manually fixing error in example delete this in future ===
    matrix[15, 8] = 1

    # === Decoding DMC ===
    decoded_data = decode_DMC(matrix)

    return decoded_data

In [ ]:
img_to_test = "../data/delete.jpg"
template_to_test = "../data/delete_template.jpg"
decode_pipeline(img_to_test, template_to_test, debug=True)